In [7]:
import numpy as np
import tkinter as tk

In [8]:
def remontee_gauss(A, b):
    # Déterminons la taille n du systeme
    n = A.shape[0]

    # Initialisation du vecteur solution x
    x = np.zeros(n, dtype=float)

    if A[n-1, n-1] == 0:
        return None, "Erreur : L'élément diagonal A[n-1, n-1] est zéro, division impossible."
    
    # Résolution du système par la méthode de remontée de Gauss
    x[n-1] = b[n-1] / A[n-1, n-1]
    for i in range(n-2, -1, -1):
        current_sum = 0.0
        for j in range(i+1, n):
            current_sum += A[i, j] * x[j]

        if A[i, i] == 0:
            return None, f"Erreur : L'élément diagonal A[{i}, {i}] est zéro, division impossible."    
        x[i] = (b[i] - current_sum) / A[i, i]

    return x, None

In [9]:
def solveur_gui():
    fenetre = tk.Tk()
    fenetre.title("Solveur de Système Linéaire Triangulaire Supérieur")
    fenetre.geometry("600x550")
    fenetre.resizable(False, False)

    # --- Variables globales pour les champs de saisie et le résultat ---
    champs_A = []
    champs_b = []
    n_global = 0 

    # Variable pour afficher les messages d'erreur ou les résultats
    message_texte = tk.StringVar()
    message_texte.set("Entrez la dimension n ci-dessus.")

    # --- Widgets d'entrée pour la taille de la matrice ---
    label_taille = tk.Label(fenetre, text="Taille de la matrice (n x n):")
    label_taille.pack(pady=10)

    entree_taille = tk.Entry(fenetre)
    entree_taille.pack(pady=5)

    # Cadre pour les messages (erreurs ou résultat)
    frame_messages = tk.Frame(fenetre, padx=10, pady=5)
    frame_messages.pack(pady=5)
    # Assurez-vous que wraplength est suffisamment grand pour vos lignes
    label_message = tk.Label(frame_messages, textvariable=message_texte, fg="black", wraplength=400, justify=tk.LEFT)
    label_message.pack()

    # --- Cadre pour la matrice A et le vecteur b (sera créé dynamiquement) ---
    frame_matrice_dynamique = None
    bouton_resoudre_widget = None

    def resoudre_systeme():
        nonlocal n_global
        A_vals = []
        b_vals = []

        # Récupérer les valeurs de A
        try:
            for i in range(n_global):
                row_vals = []
                for j in range(n_global):
                    val_str = champs_A[i][j].get()
                    if not val_str:
                        message_texte.set(f"Erreur : Veuillez remplir toutes les cellules de la matrice A. Cellule A[{i},{j}] vide.")
                        label_message.config(fg="red")
                        return
                    row_vals.append(float(val_str))
                A_vals.append(row_vals)
        except ValueError:
            message_texte.set("Erreur : Toutes les valeurs de la matrice A doivent être des nombres valides.")
            label_message.config(fg="red")
            return

        # Récupérer les valeurs de b
        try:
            for i in range(n_global):
                val_str = champs_b[i].get()
                if not val_str:
                    message_texte.set(f"Erreur : Veuillez remplir toutes les cellules du vecteur b. Cellule b[{i}] vide.")
                    label_message.config(fg="red")
                    return
                b_vals.append(float(val_str))
        except ValueError:
            message_texte.set("Erreur : Toutes les valeurs du vecteur b doivent être des nombres valides.")
            label_message.config(fg="red")
            return

        A = np.array(A_vals)
        b = np.array(b_vals)

        # Vérifier si la matrice est triangulaire supérieure (pour la méthode de remontée)
        is_upper_triangular = True
        for i in range(1, n_global):
            for j in range(i):
                if A[i, j] != 0:
                    is_upper_triangular = False
                    break
            if not is_upper_triangular:
                break

        if not is_upper_triangular:
            message_texte.set("Attention : La matrice saisie n'est pas strictement triangulaire supérieure. La méthode de remontée de Gauss peut ne pas donner un résultat correct si les éléments sous-diagonaux ne sont pas censés être nuls.")
            label_message.config(fg="orange")
        else:
            message_texte.set("") 
            label_message.config(fg="black")

        x, error_message = remontee_gauss(A, b)

        if x is not None:
            result_lines = ["Solution x:"]
            for i, val in enumerate(x):
                # Utiliser un seul \n ici pour les sauts de ligne
                result_lines.append(f"x[{i}] = {val:.4f}")
            
            # Utiliser un seul \n pour joindre les lignes
            message_texte.set("\n".join(result_lines))
            label_message.config(fg="green")
        else:
            message_texte.set(error_message)
            label_message.config(fg="red")


    def creer_champs_saisie():
        nonlocal frame_matrice_dynamique, bouton_resoudre_widget, n_global

        # 1. Nettoyer les messages précédents
        message_texte.set("")
        label_message.config(fg="black")

        # 2. Récupérer et valider n
        try:
            n_str = entree_taille.get()
            if not n_str:
                message_texte.set("Erreur : Veuillez entrer une dimension n.")
                label_message.config(fg="red")
                return

            n = int(n_str)
            if n < 1:
                message_texte.set("Erreur : La dimension n doit être un entier positif (n >= 1).")
                label_message.config(fg="red")
                return
        except ValueError:
            message_texte.set("Erreur : La dimension n doit être un nombre entier valide.")
            label_message.config(fg="red")
            return
        
        n_global = n 

        # 3. Effacer les anciens champs de saisie si ils existent
        if frame_matrice_dynamique is not None:
            frame_matrice_dynamique.destroy() 
            champs_A.clear() 
            champs_b.clear() 
            if bouton_resoudre_widget:
                bouton_resoudre_widget.destroy() 

        # 4. Créer un nouveau cadre pour la matrice A et le vecteur b
        frame_matrice_dynamique = tk.Frame(fenetre, padx=10, pady=10)
        frame_matrice_dynamique.pack(pady=10)

        # 5. Créer les Entry pour la matrice A et le vecteur b
        tk.Label(frame_matrice_dynamique, text="Matrice A").grid(row=0, column=0, columnspan=n)
        tk.Label(frame_matrice_dynamique, text="Vecteur b").grid(row=0, column=n + 1)
        
        # Pour chaque ligne i
        for i in range(n):
            # Liste temporaire pour les Entry de la ligne actuelle de A
            row_champs_A = []
            # Pour chaque colonne j de la matrice A
            for j in range(n):
                champ = tk.Entry(frame_matrice_dynamique, width=5)
                # Les éléments sous la diagonale principale sont désactivés pour une matrice triangulaire supérieure
                if j < i:
                    champ.insert(0, "0") 
                    champ.config(state="disabled", disabledbackground="lightgrey", disabledforeground="black")
                champ.grid(row=i + 1, column=j, padx=2, pady=2) 
                row_champs_A.append(champ)
            champs_A.append(row_champs_A) 

            # Label pour le signe '='
            tk.Label(frame_matrice_dynamique, text="=").grid(row=i + 1, column=n, padx=2, pady=2)

            # Entry pour le vecteur b
            champ_b = tk.Entry(frame_matrice_dynamique, width=5)
            champ_b.grid(row=i + 1, column=n + 1, padx=2, pady=2)
            champs_b.append(champ_b)

        # 6. Créer le bouton "Résoudre"
        bouton_resoudre_widget = tk.Button(fenetre, text="Résoudre le système", command=resoudre_systeme)
        bouton_resoudre_widget.pack(pady=10)
        
        message_texte.set(f"Veuillez saisir les valeurs de la matrice A (triangulaire supérieure) et du vecteur b.")


    bouton_creer_champs = tk.Button(fenetre, text="Créer les champs de saisie", command=creer_champs_saisie)
    bouton_creer_champs.pack(pady=5)
    fenetre.mainloop()


if __name__ == "__main__":
    solveur_gui()